In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
import zipfile
import os
import requests
import branca.colormap as cm
import matplotlib.colors as colors
os.getcwd()

'C:\\Users\\claud\\Documents\\GitHub\\vivienda_cdmx'

In [2]:
#Descarga y unzip

def download_unzip(url, path):
    r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'})
    with open(path, 'wb') as f:
        f.write(r.content)
    with zipfile.ZipFile(path, 'r') as z:
        z.extractall(path.replace('.zip', ''))

In [3]:
#Url de listings de airbnb en Ciudad de México
url='https://data.insideairbnb.com/mexico/df/mexico-city/2024-06-27/data/listings.csv.gz'
path_capas="capas"

In [4]:
df=pd.read_csv(url)
df['room_type']=df['room_type'].replace({'Entire home/apt':'Casa/Departamento entero','Private room':'Habitación privada','Shared room':'Habitación compartida','Hotel room':'Habitación de hotel'})
#Eliminar $ y , de price y convertir a float
df['price']=df['price'].replace({'\$':'','\,':''},regex=True).astype(float)
#Filtrar solo por Casa/Departamento entero
df=df[df['room_type']=='Casa/Departamento entero']

df

<>:4: SyntaxWarning: invalid escape sequence '\$'
<>:4: SyntaxWarning: invalid escape sequence '\,'
<>:4: SyntaxWarning: invalid escape sequence '\$'
<>:4: SyntaxWarning: invalid escape sequence '\,'
C:\Users\claud\AppData\Local\Temp\ipykernel_21804\1818334457.py:4: SyntaxWarning: invalid escape sequence '\$'
  df['price']=df['price'].replace({'\$':'','\,':''},regex=True).astype(float)
C:\Users\claud\AppData\Local\Temp\ipykernel_21804\1818334457.py:4: SyntaxWarning: invalid escape sequence '\,'
  df['price']=df['price'].replace({'\$':'','\,':''},regex=True).astype(float)


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,898633,https://www.airbnb.com/rooms/898633,20240627045222,2024-06-28,city scrape,City SkyLine View from 1 Bedroom Apt in CDMX,The LOCATION is undoubtedly THE BIG advantage ...,The hipster Roma neighborhood embodies a color...,https://a0.muscache.com/pictures/ba75b1c1-cb02...,4796027,...,4.96,4.93,4.91,NaN,f,3,2,1,0,0.41
1,35797,https://www.airbnb.com/rooms/35797,20240627045222,2024-06-27,city scrape,Villa Dante,"Dentro de Villa un estudio de arte con futon, ...","Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,153786,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
2,44616,https://www.airbnb.com/rooms/44616,20240627045222,2024-06-28,city scrape,CONDESA HAUS B&B,A new concept of hosting in mexico through a b...,NaN,https://a0.muscache.com/pictures/251410/ec75fe...,196253,...,4.78,4.98,4.48,NaN,f,13,4,2,0,0.42
3,913084,https://www.airbnb.com/rooms/913084,20240627045222,2024-06-28,city scrape,15 blocks from Paseo de la Reforma,NaN,"It borders the old and traditional San Rafael,...",https://a0.muscache.com/pictures/16663562/7bed...,4647293,...,4.93,4.46,4.75,NaN,t,1,1,0,0,1.19
4,67703,https://www.airbnb.com/rooms/67703,20240627045222,2024-06-28,city scrape,"2 bedroom apt. deco bldg, Condesa","Comfortably furnished, sunny, 2 bedroom apt., ...",NaN,https://a0.muscache.com/pictures/3281720/6f078...,334451,...,4.92,4.98,4.91,NaN,f,2,2,0,0,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26531,1187191423076770832,https://www.airbnb.com/rooms/1187191423076770832,20240627045222,2024-06-28,city scrape,¡Crystal! - 7BR Fantastic! In heart of Roma,Located in the vibrant neighborhood of Roma Su...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,19255745,...,NaN,NaN,NaN,NaN,t,8,8,0,0,NaN
26532,1187248376951514656,https://www.airbnb.com/rooms/1187248376951514656,20240627045222,2024-06-28,city scrape,Iconic Suite | Terrace | View !,"Welcome to Icon Cibles, a newly built modern s...",NaN,https://a0.muscache.com/pictures/hosting/Hosti...,7681206,...,NaN,NaN,NaN,NaN,f,11,11,0,0,NaN
26533,1187269835146549014,https://www.airbnb.com/rooms/1187269835146549014,20240627045222,2024-06-28,city scrape,Acogedor y moderno depa en la Condesa,Enjoy the warmth and tranquility provided by t...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,50951150,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
26534,1187283698786191137,https://www.airbnb.com/rooms/1187283698786191137,20240627045222,2024-06-28,city scrape,Estudio con amplia terraza,Private studio on the top floor of an old hous...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,147254250,...,NaN,NaN,NaN,NaN,f,3,3,0,0,NaN


In [5]:
url_datos_planos = "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/ageb_manzana/RESAGEBURB_09_2020_csv.zip"
#download_unzip(url_datos_planos, 'capas/RESAGEBURB_09_2020_csv.zip')

In [6]:
url_mza="https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/09_ciudaddemexico.zip"
#download_unzip(url_mza, 'capas/capas.zip')

In [7]:
planos=pd.read_csv('capas/RESAGEBURB_09_2020_csv/RESAGEBURB_09CSV20.csv').rename(columns=str.lower)
planos['cvegeo'] = planos['entidad'].astype(str).str.zfill(2) + planos['mun'].astype(str).str.zfill(3) + planos['loc'].astype(str).str.zfill(4) + planos['ageb'].astype(str).str.zfill(4) + planos['mza'].astype(str).str.zfill(3)
planos=planos[planos['mun']!=0]
#Eliminar agregados de municipio. 
planos=planos[planos['loc']!=0]
#Eliminar agregados de localidad.
planos=planos[planos['ageb']!='0000']
#Eliminar agregados de manzana.
planos=planos[planos['mza']!=0]
#Dejar solo cvegeo y tvivparhab
planos=planos[['cvegeo','pobtot','tvivparhab','nom_mun']]
#pobtot y tvivparhab a int
planos['pobtot']=pd.to_numeric(planos['pobtot'],errors='coerce')
planos['tvivparhab']=pd.to_numeric(planos['tvivparhab'],errors='coerce')
planos


,cvegeo,pobtot,tvivparhab,nom_mun
4,0900200010010001,159,48.0,Azcapotzalco
5,0900200010010002,145,42.0,Azcapotzalco
6,0900200010010003,124,32.0,Azcapotzalco
7,0900200010010004,158,42.0,Azcapotzalco
8,0900200010010005,154,45.0,Azcapotzalco
...,...,...,...,...
68936,0901700011524021,106,31.0,Venustiano Carranza
68937,0901700011524022,107,40.0,Venustiano Carranza
68938,0901700011524023,246,73.0,Venustiano Carranza
68939,0901700011524024,158,47.0,Venustiano Carranza


In [8]:
#Capa de manzanas
manzanas=gpd.read_file('capas/capas/conjunto_de_datos/09m.shp').rename(columns=str.lower).to_crs(epsg=4326)
len(manzanas)

66789

In [9]:
#Pegar datos a manzanas
manzanas=pd.merge(manzanas,planos,on='cvegeo',how='left')
print(len(manzanas))
manzanas=manzanas[['cvegeo','nom_mun','geometry','pobtot','tvivparhab']]
type(manzanas)

66789


geopandas.geodataframe.GeoDataFrame

In [10]:
#Asociar manzanas a puntos de airbnb
gdf=gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df['longitude'],df['latitude']),crs=4326)
gdf

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,geometry
0,898633,https://www.airbnb.com/rooms/898633,20240627045222,2024-06-28,city scrape,City SkyLine View from 1 Bedroom Apt in CDMX,The LOCATION is undoubtedly THE BIG advantage ...,The hipster Roma neighborhood embodies a color...,https://a0.muscache.com/pictures/ba75b1c1-cb02...,4796027,...,4.93,4.91,NaN,f,3,2,1,0,0.41,POINT (-99.16586 19.42063)
1,35797,https://www.airbnb.com/rooms/35797,20240627045222,2024-06-27,city scrape,Villa Dante,"Dentro de Villa un estudio de arte con futon, ...","Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,153786,...,NaN,NaN,NaN,f,1,1,0,0,NaN,POINT (-99.27178 19.38283)
2,44616,https://www.airbnb.com/rooms/44616,20240627045222,2024-06-28,city scrape,CONDESA HAUS B&B,A new concept of hosting in mexico through a b...,NaN,https://a0.muscache.com/pictures/251410/ec75fe...,196253,...,4.98,4.48,NaN,f,13,4,2,0,0.42,POINT (-99.17794 19.41162)
3,913084,https://www.airbnb.com/rooms/913084,20240627045222,2024-06-28,city scrape,15 blocks from Paseo de la Reforma,NaN,"It borders the old and traditional San Rafael,...",https://a0.muscache.com/pictures/16663562/7bed...,4647293,...,4.46,4.75,NaN,t,1,1,0,0,1.19,POINT (-99.17263 19.43956)
4,67703,https://www.airbnb.com/rooms/67703,20240627045222,2024-06-28,city scrape,"2 bedroom apt. deco bldg, Condesa","Comfortably furnished, sunny, 2 bedroom apt., ...",NaN,https://a0.muscache.com/pictures/3281720/6f078...,334451,...,4.98,4.91,NaN,f,2,2,0,0,0.32,POINT (-99.16857 19.41152)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26531,1187191423076770832,https://www.airbnb.com/rooms/1187191423076770832,20240627045222,2024-06-28,city scrape,¡Crystal! - 7BR Fantastic! In heart of Roma,Located in the vibrant neighborhood of Roma Su...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,19255745,...,NaN,NaN,NaN,t,8,8,0,0,NaN,POINT (-99.16825 19.40455)
26532,1187248376951514656,https://www.airbnb.com/rooms/1187248376951514656,20240627045222,2024-06-28,city scrape,Iconic Suite | Terrace | View !,"Welcome to Icon Cibles, a newly built modern s...",NaN,https://a0.muscache.com/pictures/hosting/Hosti...,7681206,...,NaN,NaN,NaN,f,11,11,0,0,NaN,POINT (-99.16626 19.42057)
26533,1187269835146549014,https://www.airbnb.com/rooms/1187269835146549014,20240627045222,2024-06-28,city scrape,Acogedor y moderno depa en la Condesa,Enjoy the warmth and tranquility provided by t...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,50951150,...,NaN,NaN,NaN,f,1,1,0,0,NaN,POINT (-99.17099 19.40865)
26534,1187283698786191137,https://www.airbnb.com/rooms/1187283698786191137,20240627045222,2024-06-28,city scrape,Estudio con amplia terraza,Private studio on the top floor of an old hous...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,147254250,...,NaN,NaN,NaN,f,3,3,0,0,NaN,POINT (-99.16645 19.41237)


In [11]:
joined = gpd.sjoin(gdf, manzanas, predicate='within')
#Contar cuantos puntos de airbnb hay en cada manzana
joined=joined.groupby('cvegeo').size().reset_index(name='viviendas_airbnb')
joined

,cvegeo,viviendas_airbnb
0,0900200010186031,3
1,0900200010190007,2
2,0900200010190024,1
3,0900200010218002,1
4,0900200010275034,1
...,...,...
4387,0901700011416006,1
4388,0901700011416011,1
4389,0901700011420004,1
4390,0901700011435001,1


In [12]:
#Pegar datos a manzanas
manzanas=pd.merge(manzanas,joined,on='cvegeo',how='left')
#Sustituir NaN por 0 en viviendas_airbnb
manzanas['viviendas_airbnb']=manzanas['viviendas_airbnb'].fillna(0)
manzanas["porcentaje_airbnb"]=manzanas["viviendas_airbnb"]/manzanas["tvivparhab"]*100
manzanas['porcentaje_airbnb']=manzanas['porcentaje_airbnb'].fillna(0)

#Si porcentaje_airbnb es inf poner 0
manzanas.loc[manzanas['porcentaje_airbnb'] == np.inf, 'porcentaje_airbnb'] = 0
#Si porcentaje es mayor a 100, poner 100
manzanas.loc[manzanas['porcentaje_airbnb'] > 100, 'porcentaje_airbnb'] = 100
#Eliminar manzanas con 0
manzanas=manzanas[manzanas['porcentaje_airbnb']!=0]
#Porcentaje de airbnb con un decimal
manzanas['porcentaje_airbnb']=manzanas['porcentaje_airbnb'].round(1)

print(max(manzanas['porcentaje_airbnb']))
print(min(manzanas['porcentaje_airbnb'])) 
manzanas

100.0
0.1


,cvegeo,nom_mun,geometry,pobtot,tvivparhab,viviendas_airbnb,porcentaje_airbnb
59,0901000011881046,Álvaro Obregón,"POLYGON ((-99.25252 19.34898, -99.25261 19.349...",99.0,34.0,2.0,5.9
71,0901000011720040,Álvaro Obregón,"POLYGON ((-99.28857 19.32274, -99.28873 19.322...",608.0,145.0,1.0,0.7
91,0901000011595004,Álvaro Obregón,"POLYGON ((-99.24578 19.36750, -99.24570 19.367...",226.0,43.0,1.0,2.3
141,0901000011190007,Álvaro Obregón,"POLYGON ((-99.19833 19.34303, -99.19759 19.343...",268.0,75.0,1.0,1.3
142,0901000011186017,Álvaro Obregón,"POLYGON ((-99.20839 19.34496, -99.20832 19.345...",334.0,117.0,1.0,0.9
...,...,...,...,...,...,...,...
65584,0900700015732001,Iztapalapa,"POLYGON ((-99.04437 19.32248, -99.04436 19.322...",697.0,173.0,2.0,1.2
65625,0900700015446007,Iztapalapa,"POLYGON ((-99.10968 19.36840, -99.10968 19.368...",139.0,39.0,1.0,2.6
65698,0900700010500003,Iztapalapa,"POLYGON ((-99.12964 19.36981, -99.12960 19.370...",584.0,164.0,1.0,0.6
66189,0900700015380001,Iztapalapa,"POLYGON ((-99.04596 19.31830, -99.04562 19.318...",1584.0,407.0,1.0,0.2


In [13]:
print("Total de viviendas airbnb: ",manzanas['viviendas_airbnb'].sum())

Total de viviendas airbnb:  12388.0


In [14]:


# Mapa inicial centrado en cdmx
m = folium.Map(
    location=[19.4326, -99.1332],
    zoom_start=13,
    tiles=None
)

# Capa maps
google_hybrid = folium.TileLayer(
    
    tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
    name='Google Maps',
    attr="Fuente: Elaborado con información de INEGI. Censo de Población y Vivienda 2020 y de Inside Airbnb",
    overlay=False,
    control=True
)
google_hybrid.add_to(m)

#Colores para la barra de color
colormap = colors.LinearSegmentedColormap.from_list('YlOrRd', ['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'])

# Obtener valores máximos y mínimos
min_value = manzanas['porcentaje_airbnb'].min()
max_value = manzanas['porcentaje_airbnb'].max()

# Define a branca colormap to be used as a color bar
color_scale = cm.LinearColormap(['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'],
                                vmin=min_value, vmax=max_value, caption="Porcentaje de viviendas Airbnb")


def style_function(feature):
    porcentaje = feature['properties']['porcentaje_airbnb']
    if porcentaje is None:
        color = '#ffffff'  
    else:
        normalized_value = (porcentaje - min_value) / (max_value - min_value)
        color = colors.to_hex(colormap(normalized_value))
    
    return {
        'fillColor': color,
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    }

# Puntos airbnb
airbnb_layer = folium.FeatureGroup(name='Vivienda Airbnb', overlay=True, show=False)
for i, row in gdf.iterrows():
    tooltip_text = f"""
    <strong>Nombre:</strong> {row['name']}<br>
    <strong>Precio:</strong> ${row['price']} por noche<br>
    <strong>Capacidad:</strong> {row['accommodates']} personas<br>
    <strong>Baños:</strong> {row['bathrooms']}<br>
    <strong>Habitaciones:</strong> {row['bedrooms']}
    """
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color='blue',
        fill=True,
        fill_opacity=0.6,
        tooltip=tooltip_text
    ).add_to(airbnb_layer)
airbnb_layer.add_to(m)

folium.GeoJson(
    manzanas,
    name='Porcentaje de viviendas Airbnb',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['cvegeo','nom_mun', 'porcentaje_airbnb'],
                                  aliases=['Clave de manzana','Alcaldía', 'Vivienda Airbnb/Total (%)'],
                                  style=("background-color: white; color: black; font-family: 'Century Gothic'; font-size: 12px; padding: 5px;")
                                  ),
).add_to(m)

# Barra de color
color_scale.add_to(m)

# Control de capas
folium.LayerControl(collapsed=False).add_to(m)

m.save('mapa_airbnb.html')
